In [1]:
import pandas as pd
from dhis2 import Api

from src.main.common import FileUtil
from src.main.common import LoggingConfig
from src.main.application.income import *
from src.main.application.service import *
from src.infrastructure.forms import *
from src.infrastructure.adapters import *
from src.main.application.out import *

In [2]:
credentials = FileUtil.load_credentias()
username = credentials['username']
password = credentials['password']

url = 'https://dhis-ao.icap.columbia.edu'
api = Api(url, username, password)

In [3]:
# Get the list of supported facilities
org_units = api.get('organisationUnitGroups/gH2DlwAo1ja', params={'fields':'organisationUnits'}).json()['organisationUnits']

In [4]:
# period of analysis
start_period = '2024-04-01'
end_period = '2024-06-30'
period = '2024Q2'

In [5]:
# Logger
config = FileUtil.load_logging_config()
logging_config = LoggingConfig(config)
logger = logging_config.logging_setup()

In [6]:
enrollments = pd.DataFrame(columns=['enrollment', 'trackedEntity', 'program', 'status', 'orgUnit', 'enrolledAt', 'patientIdentifier', 'patientAge', 'patientSex','patientName', 'ancType', 'testResult', 'artStartDate', 'artStatus', 'onArt'])
enrollments.to_csv('PMTCT_ENROLLMENTS.csv', index=False)

In [7]:
# Load patients enrolled
patient_demographics = PatientDemographicForm(api)
patient_events = PatientEventForm(api)

for org_unit in org_units:

    org_unit = org_unit['id']

    #get all patient enrollments
    patients_enrolled = api.get('tracker/enrollments', params={'orgUnit':org_unit, 'skipPaging':'true', 'program': PTV, 'fields':'{,enrollment, enrolledAt, orgUnit, trackedEntity, program, status,}', 'enrolledAfter':f'{start_period}', 'enrolledBefore':f'{end_period}', 'order':'enrolledAt:desc'})
    patients_enrolled = patients_enrolled.json()['instances']

    logger.info(f"Processing the facility: {org_unit}, a total of {len(patients_enrolled)} enrolled patient(s)")
    
    counter = 1

    for patient_enrolled in patients_enrolled:
        patient_id = patient_enrolled['trackedEntity']

        patient_demographics.add_demographics(patient_enrolled)
        patient_events.add_patient_first_anc_event(patient_enrolled)

        logger.info(f"From {len (patients_enrolled)} patietnts enrolled, {counter} (is) are ready to be processed.")
        counter = counter + 1 

    if patients_enrolled:
        patients_enrolled = pd.json_normalize(patients_enrolled)
        
        enrollments = pd.read_csv('PMTCT_ENROLLMENTS.csv')
        enrollments = pd.concat([enrollments, patients_enrolled])
        enrollments.to_csv('PMTCT_ENROLLMENTS.csv', index=False, encoding='utf-8')

In [8]:
dpi_enrollments = pd.DataFrame(columns=['enrollment', 'trackedEntity', 'program', 'status', 'orgUnit', 'enrolledAt', 'patientIdentifier', 'patientAge', 'patientSex','patientName', 'exposed', 'pcrNumber', 'testResult', 'artStartDate'])
dpi_enrollments.to_csv('DPI_ENROLLMENTS.csv', index=False)

# Load patients enrolled
patient_demographics = PatientDemographicForm(api)
patient_events = PatientEventForm(api)

for org_unit in org_units:

    org_unit = org_unit['id']

    #get all patient enrollments
    patients_enrolled = api.get('tracker/enrollments', params={'orgUnit':org_unit, 'skipPaging':'true', 'program': DPI, 'fields':'{,enrollment, enrolledAt, orgUnit, trackedEntity, program, status,}', 'enrolledAfter':f'{start_period}', 'enrolledBefore':f'{end_period}', 'order':'enrolledAt:asc'})
    patients_enrolled = patients_enrolled.json()['instances']

    logger.info(f"Processing the facility: {org_unit}, a total of {len(patients_enrolled)} enrolled patient(s)")
    
    counter = 1

    for patient_enrolled in patients_enrolled:
        patient_id = patient_enrolled['trackedEntity']

        patient_demographics.add_demographics(patient_enrolled)
        patient_events.add_patient_last_dpi_event(patient_enrolled)

        logger.info(f"From {len (patients_enrolled)} patietnts enrolled, {counter} (is) are ready to be processed.")
        counter = counter + 1 

    if patients_enrolled:
        patients_enrolled = pd.json_normalize(patients_enrolled)
        
        enrollments = pd.read_csv('DPI_ENROLLMENTS.csv')
        enrollments = pd.concat([enrollments, patients_enrolled])
        enrollments.to_csv('DPI_ENROLLMENTS.csv', index=False, encoding='utf-8')

In [6]:
enrollments = pd.read_csv('PMTCT_ENROLLMENTS.csv')
enrollments = enrollments.to_dict(orient='records')

pmtct_art_numerator_service = ComputePmtctArtNumeratorService()
pmtct_art_numerator_patients = pmtct_art_numerator_service.compute(enrollments)
pmtct_art_numerator_indicator_metadata_port = PmtctArtNumeratorIndicatorMetadataAdapter(api)
pmtct_art_numerator_disaggegation_service = ComputePmtctArtNumeratorDisaggregationService(logger, pmtct_art_numerator_indicator_metadata_port)
pmtct_art_numerator_patients_disaggregation = pmtct_art_numerator_disaggegation_service.compute(pmtct_art_numerator_patients, end_period)

pmtct_stat_denominator_service = ComputePmtctStatDenominatorService()
pmtct_stat_denominator_patients = pmtct_stat_denominator_service.compute(enrollments)
pmtct_stat_denominator_indicator_metadata_port = PmtctStatDenominatorIndicatorMetadataAdapter(api)
pmtct_stat_denominator_disaggregation_service = ComputePmtctStatDenominatorDisaggregationService(logger, pmtct_stat_denominator_indicator_metadata_port)
pmtct_stat_denominator_patients_disaggregation = pmtct_stat_denominator_disaggregation_service.compute(pmtct_stat_denominator_patients, end_period)

pmtct_stat_numerator_service = ComputePmtctStatNumeratorService()
pmtct_stat_numerator_patients = pmtct_stat_numerator_service.compute(enrollments)
pmtct_stat_numerator_indicator_metadata_port = PmtctStatNumeratorIndicatorMetadataAdapter(api)
pmtct_stat_numerator_disaggregation_service = ComputePmtctStatNumeratorDisaggregationService(logger, pmtct_stat_numerator_indicator_metadata_port)
pmtct_stat_numerator_patients_disaggregation = pmtct_stat_numerator_disaggregation_service.compute(pmtct_stat_numerator_patients, end_period)

dpi_enrollments = pd.read_csv('DPI_ENROLLMENTS.csv')
dpi_enrollments = dpi_enrollments.to_dict(orient='records')

pmtct_eid_use_case: ComputePmtctEidUseCase = ComputePmtctEidService()
pmtct_eid_patients = pmtct_eid_use_case.compute(dpi_enrollments)
pmtct_eid_indicators_metadata_port: IndicatorMetadataPort = PmtctEidIndicatorMedatadaAdapter(api)
pmtct_eid_use_case: ComputePmtctEidDisaggregationUseCase = ComputePmtctEidDisaggregationService(logger, pmtct_eid_indicators_metadata_port)
pmtct_eid_patients_disaggregation = pmtct_eid_use_case.compute(pmtct_eid_patients, end_period)

pmtct_hei_use_case: ComputePmtctHeiUseCase = ComputePmtctHeiService()
pmtct_hei_patients = pmtct_hei_use_case.compute(dpi_enrollments)
pmtct_hei_indicators_metadata_port: IndicatorMetadataPort = PmtctHeiIndicatorsMetadataAdapter(api)
pmtct_hei_pos_indicators_metadata_port: IndicatorMetadataPort = PmtctHeiPosIndicatorsMetadataAdapter(api)

pmtct_hei_disaggregation_use_case: ComputePmtctHeiDisaggregationUseCase = ComputePmtctHeiDisaggregationService(logger, pmtct_hei_indicators_metadata_port, pmtct_hei_pos_indicators_metadata_port)
pmtct_hei_patients_disaggregation = pmtct_hei_disaggregation_use_case.compute(pmtct_hei_patients, end_period)

combination = pmtct_art_numerator_patients_disaggregation + pmtct_stat_denominator_patients_disaggregation + pmtct_stat_numerator_patients_disaggregation + pmtct_eid_patients_disaggregation + pmtct_hei_patients_disaggregation

if combination:
    # extract data
    indicators = pd.json_normalize(combination)
    indicators['period'] = period
    indicators = indicators[['dataElement','period','orgUnit', 'categoryOptionCombo','attributeOptionCombo', 'value']]
    indicators = indicators.sort_values(['orgUnit', 'dataElement', 'period'])
    indicators.to_csv('PMTCT_DATA.csv', index=False)

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 0105-09-27, at position 0